In [ ]:
import sys
import requests # pip install requests
import lxml.etree as ET # pip install lxml
from PyQt5.QtWidgets import (QWidget, QLabel, QApplication, QComboBox, QPushButton, QLineEdit)
from PyQt5.QtGui import QPixmap, QFont


class CBR_API(QWidget):

    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):

        self.val()
        self.su()

        ok_button = QPushButton('OK', self)
        ok_button.resize(50, 23)
        ok_button.move(220, 198)

        ok_button.clicked.connect(self.make_request)

        self.load_result()

        self.setFixedSize(300, 400)
        self.setWindowTitle('Конвертер')
        self.show()

    def val(self):
        self.val_combo = QComboBox(self)

        val_label = QLabel('Валюта', self)
        val_label.move(20, 170)

        self.val_combo.addItem('RUB')
        self.val_combo.addItem('EUR')
        self.val_combo.addItem('USD')

        self.val_combo.move(20, 200)


    def su(self):
        self.su_text = QLineEdit(self)
        self.su_text.resize(110, 20)
        self.su_text.move(100, 200)

        su_label = QLabel('Сумма:', self)
        su_label.move(127, 170)

    def make_request(self):
        val_value = self.val_combo.currentText()
        su = self.su_text.text()

        try:
            su = int(su)
        except ValueError:
            QMessageBox.warning(self, 'Error' .format(su))
            su = 0
        
        result = self.get_result(val_value, su)
        
        self.ruble_value.setText(f"{result['RUB']} RUB")
        self.ruble_value.adjustSize()

        self.dollar_value.setText(f"{result['USD']} USD")
        self.dollar_value.adjustSize()

        self.euro_value.setText(f"{result['EUR']} EUR")
        self.euro_value.adjustSize()
        
    def get_result(self, val, su):
        
        result = {
            'RUB': 0,
            'USD': 0,
            'EUR': 0,
        }

        day = '06'
        month = '05'
        year = '2020'

        try:
            # request: 'http://www.cbr.ru/scripts/XML_daily.asp?date_req=ДД/ММ/ГГГ'
            get_xml = requests.get(f'http://www.cbr.ru/scripts/XML_daily.asp?date_req={day}/{month}/{year}')

            '''response: 
                <ValCurs Date="22.04.2017" name="Foreign Currency Market">
                <Valute ID="R01235">
                    <NumCode>840</NumCode>
                    <CharCode>USD</CharCode>
                    <Nominal>1</Nominal>
                    <Name>Доллар США</Name>
                    <Value>56,2307</Value>
                </Valute>
            
                <Valute ID="R01239">
                    <NumCode>978</NumCode>
                    <CharCode>EUR</CharCode>
                    <Nominal>1</Nominal>
                    <Name>Евро</Name>
                    <Value>60,3187</Value>
                </Valute>
            </ValCurs>'''

            structure = ET.fromstring(get_xml.content)

        except:
            return result

        try:
            dollar = structure.find("./*[@ID='R01235']/Value")
            result['USD'] = dollar.text.replace(',', '.')
        except:
            result['USD'] = 'x'

        try:
            euro = structure.find("./*[@ID='R01239']/Value")
            result['EUR'] = euro.text.replace(',', '.')
        except:
            result['EUR'] = 'x'
        
        if val == 'RUB':
            result['RUB'] = "%.2f" % su
            result['USD'] = "%.2f" % (su / float(result['USD']))
            result['EUR'] = "%.2f" % (su / float(result['EUR']))

        elif val == 'EUR':
            result['RUB'] = "%.2f" % (su * float(result['EUR']))
            result['USD'] = "%.2f" % (float(result['RUB']) / float(result['USD']))
            result['EUR'] = "%.2f" % su 

        elif val == 'USD':
            result['RUB'] = "%.2f" % (su * float(result['USD']))
            result['EUR'] = "%.2f" % (float(result['RUB']) / float(result['EUR']))
            result['USD'] = "%.2f" % su 

        return result

    
    def load_result(self):

        font = QFont()
        font.setFamily('Arial')
        font.setPointSize(18)

        self.ruble_value = QLabel('0 RUB', self)
        self.ruble_value.setFont(font)
        self.ruble_value.move(80, 250)

        self.dollar_value = QLabel('0 EUR', self)
        self.dollar_value.setFont(font)
        self.dollar_value.move(80, 300)

        self.euro_value = QLabel('0 USD', self)
        self.euro_value.setFont(font)
        self.euro_value.move(80, 350)


def main():
    app = QApplication(sys.argv)
    money = CBR_API()

    sys.exit(app.exec_())


if __name__ == '__main__':
    main()